<a href="https://colab.research.google.com/github/SDGPTeamX/PregRed/blob/ml_umeshika/CreatingEndpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
import boto3, re
from sagemaker import get_execution_role
from sagemaker.tensorflow.model import TensorFlowModel

# giving path to the dataset in SageMaker
MODEL_LOCATION ='Pregnancy_Prediction_Model.h5'

# load the model
model = load_model(MODEL_LOCATION, compile = False)

In [ ]:
# Convert pre-trained keras model into a file structure which AWS expects
def convert_h5_to_aws(loaded_model):
    # the file structure which AWS expects
    model_version = '1'
    export_dir = 'export/Servo/' + model_version

    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)

    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
    
    from tensorflow.keras import backend as K
    with K.get_session() as sess:
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()

    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)

convert_h5_to_aws(model)

In [ ]:
# Move the tarball (tar.gz) to S3
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [ ]:
#create an empty python file
!touch train.py

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

# Create a Sagemaker model (see AWS console>SageMaker>Models)
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

In [ ]:
# Deploy a SageMaker to an endpoint
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')